In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import preprocess_kgptalkie as ps
import seaborn as sns
import gensim
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [2]:
fake = pd.read_csv('F:\\Lecture notes\\Sem-5\\MP-III\\Dataset\\Fake.csv')
real = pd.read_csv('F:\\Lecture notes\\Sem-5\\MP-III\\Dataset\\True.csv')

In [3]:
unknown_publishers = []
for index, row in enumerate(real.text.values):
    try:
        record = row.split('-', maxsplit = 1)
        record[1]
        assert(len(record[0])<120)
    except:
        unknown_publishers.append(index)

In [4]:
real = real.drop(8970, axis = 0)

In [5]:
publisher = []
tmp_text = []

for index, row in enumerate(real.text.values):
    if index in unknown_publishers:
        tmp_text.append(row)
        publisher.append('Unknown')

    else:
        record = row.split('-', maxsplit = 1)
        publisher.append(record[0].strip())
        tmp_text.append(record[1].strip())

In [6]:
real['publisher'] = publisher
real['text'] = tmp_text

In [7]:
empty_fake_index = [ index for index,text in enumerate (fake.text.tolist()) if str(text).strip() == ""]

In [8]:
real['text'] = real['title'] + " " + real['text']
fake['text'] = fake['title'] + " " + fake['text']

In [9]:
real['text'] = real['text'].apply(lambda x: str(x).lower())
fake['text'] = fake['text'].apply(lambda x: str(x).lower())

In [10]:
real['class'] = 1
fake['class'] = 0

In [11]:
real = real[['text', 'class']]
fake = fake[['text', 'class']]
data = real.append(fake, ignore_index=True)

In [12]:
data['text'] = data['text'].apply(lambda x: ps.remove_special_chars(x))

In [13]:
y = data['class'].values
X = [d.split() for d in data['text'].tolist()]
DIM = 100
w2v_model = gensim.models.Word2Vec(sentences=X, window=10, min_count=1)

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

In [15]:
X = tokenizer.texts_to_sequences(X)

In [16]:
max_len = 1000
X = pad_sequences(X, maxlen = max_len)

In [17]:
vocab_size = len(tokenizer.word_index) + 1
vocab = tokenizer.word_index

In [18]:
def get_weight_matrix(model):
    weight_matrix = np.zeros((vocab_size, 100))
    
    for word, i in vocab.items():
        weight_matrix[i] = model.wv[word]
    
    return weight_matrix

In [19]:
embedding_vectors = get_weight_matrix(w2v_model)

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim = 100, weights = [embedding_vectors], input_length = max_len, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
model.fit(X_train, y_train, validation_split = 0.3, epochs=6)

Epoch 1/6
737/737 [==============================] - 127s 145ms/step - loss: 0.2076 - acc: 0.9176 - val_loss: 0.1367 - val_acc: 0.9496
Epoch 2/6
737/737 [==============================] - 149s 203ms/step - loss: 0.0831 - acc: 0.9710 - val_loss: 0.0629 - val_acc: 0.9788
Epoch 3/6
737/737 [==============================] - 188s 256ms/step - loss: 0.0356 - acc: 0.9889 - val_loss: 0.0448 - val_acc: 0.9859
Epoch 4/6
737/737 [==============================] - 188s 255ms/step - loss: 0.0368 - acc: 0.9880 - val_loss: 0.0260 - val_acc: 0.9924
Epoch 5/6
737/737 [==============================] - 212s 287ms/step - loss: 0.0122 - acc: 0.9967 - val_loss: 0.0259 - val_acc: 0.9922
Epoch 6/6
737/737 [==============================] - 193s 262ms/step - loss: 0.0076 - acc: 0.9978 - val_loss: 0.0238 - val_acc: 0.9937


In [ ]:
y_pred = (model.predict(X_test) >= 0.5).astype(int)

In [ ]:
x = input("Enter News to check for Real or Fake")

In [ ]:
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x, maxlen = max_len)

In [ ]:
news = (model.predict(x)>=0.5).astype(int)

In [ ]:
if news == 1:
    print("It's a Real News")
else:
    print("It's a Fake News")